# PROJETO ETL - Análise do desempenho dos alunos do Ceará no ENEM 2019

## PARTE I - Extraindo os dados

In [ ]:
#Instalação/Aualizaç da bilbioteca dask para gerenciamento de datframe de grandes tamanhos. Será Utilizada ao invés do Pandas 
#devido ao tamanho do arquivo .csv
!pip install dask
!pip install --upgrade pandas

### 1.0 - Importação de bibliotecas Úteis

In [1]:
#importação de Bibliotecas
import dask.dataframe as dd
import pandas as pd
import zipfile
import requests
import os
from io import BytesIO

### 2.0 - Preparação do Ambiente (Criação de Pasta) e Efetuando Download do Arquivo do Site do Inep

In [4]:
#Criar estrutura de pastas para armazenamento do arquivo baixado
os.makedirs('./enem2019', exist_ok=True)

In [5]:
#Baixando o arquivo do link abaixo e descompactando na pasta /enem2019
#URL dos dados
url = 'https://download.inep.gov.br/microdados/microdados_enem_2019.zip'
#Baixando o Arquivo .ZIP
print('Baixando o Arquivo....')
filebytes = BytesIO(
    requests.get(url).content
)
#Descompatando na pasta destino
print('Download Concluído. Aguarde a descompactação....')
myzip = zipfile.ZipFile(filebytes).extractall('./enem2019')
print('Processamento Concluído.')

### 3.0 - Lendo os dados em lote (Arquivo Muito Grande) e colocando em um dataframe já Filtrado por Estado = Ceará

In [2]:
# Cria dataframe com mais memória devido ao tamanho do arquivo
enem = dd.read_csv(
    './enem2019/DADOS/microdados_enem_2019.csv',
    sep = ';' , 
    decimal=',' ,
    dtype={'CO_MUNICIPIO_NASCIMENTO': 'float64',
            'CO_UF_NASCIMENTO': 'float64',
            'NU_IDADE': 'float64',
            'NO_MUNICIPIO_ESC': 'object',
            'SG_UF_ESC': 'object'},
    encoding='latin1'
)

In [3]:
#Recriando o Dataframe Só para Ceará = Processo de Filtragem
enem = enem.loc[enem.SG_UF_RESIDENCIA == 'CE']

## PARTE II - Transformando os dados

In [4]:
#Recriando o Dataframe só com as colunas necessárias
enem_ce = enem[['NU_INSCRICAO',         
                'SG_UF_RESIDENCIA',     
                'TP_SEXO',
                'TP_COR_RACA',
                'TP_ESCOLA',
                'TP_ENSINO',
                'NU_NOTA_CN',
                'NU_NOTA_MT',
                'Q001',
                'Q002',
                'Q004',
                'Q005',
                'Q006']].compute()

In [7]:
#Analisando Quantidade de Nulos no Dataframe
enem_ce.isnull().sum()

NU_INSCRICAO            0
SG_UF_RESIDENCIA        0
TP_SEXO                 0
TP_COR_RACA             0
TP_ESCOLA               0
TP_ENSINO           75865
NU_NOTA_CN              0
NU_NOTA_MT              0
Q001                    0
Q002                    0
Q004                    0
Q005                    0
Q006                    0
dtype: int64

In [8]:
#Cria backuo do DataFrame
Enem_bkp = enem_ce

In [6]:
#Apagando as Linhas com Notas Zeradas - Excluindos Alunos Faltantes
enem_ce = enem_ce.dropna(subset=['NU_NOTA_MT'])

In [9]:
#Quantidade de Linhas
enem_ce.shape[0]

222780

In [40]:
#Verificando os Tipos de Dados do Dataframe
enem_ce.dtypes

NU_INSCRICAO          int64
SG_UF_RESIDENCIA     object
TP_SEXO              object
TP_COR_RACA           int64
TP_ESCOLA             int64
TP_ENSINO           float64
NU_NOTA_CN          float64
NU_NOTA_MT          float64
Q001                 object
Q002                 object
Q004                 object
Q005                  int64
Q006                 object
dtype: object

In [11]:
#Atualizando Tipos de Dados de Colunas Importantes
enem_ce['NU_NOTA_CN'] = enem_ce['NU_NOTA_CN'].astype('float')
enem_ce['NU_NOTA_MT'] = enem_ce['NU_NOTA_MT'].astype('float')

In [13]:
#Analise detalhada do Dataframe
enem_ce.describe()

,NU_INSCRICAO,TP_COR_RACA,TP_ESCOLA,TP_ENSINO,NU_NOTA_CN,NU_NOTA_MT,Q005
count,2.227800e+05,222780.000000,222780.000000,146915.000000,222780.000000,222780.000000,222780.000000
mean,1.900030e+11,2.443016,1.461379,1.002001,468.256969,517.529065,4.105333
std,1.408311e+06,0.956225,0.573180,0.044690,75.815679,109.034242,1.450371
min,1.900010e+11,0.000000,1.000000,1.000000,0.000000,0.000000,1.000000
25%,1.900019e+11,2.000000,1.000000,1.000000,409.300000,431.900000,3.000000
50%,1.900028e+11,3.000000,1.000000,1.000000,457.500000,492.700000,4.000000
75%,1.900042e+11,3.000000,2.000000,1.000000,520.800000,587.800000,5.000000
max,1.900061e+11,5.000000,3.000000,2.000000,841.800000,985.500000,20.000000


### Retirando alguns Insights

In [24]:
#Percentual dde Participação por Sexo DEclarado o Enem no Ceará
round((enem_ce.TP_SEXO.value_counts() / enem_ce.shape[0] ) * 100,2)

F    56.11
M    43.89
Name: TP_SEXO, dtype: float64

In [17]:
#Média geral de Matemática dos alunos Cearenses
print('Média Geral de Matemática: ', round(enem_ce['NU_NOTA_MT'].mean(),2))

Média Geral de Matematica:  517.53


In [21]:
#Média geral de Matemática dos alunos de Escola Particular 
print('Média Geral de Matemática (Particular): ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_ESCOLA == 3].mean(),2))

Média Geral de Matemática (Particular):  625.95


In [28]:
#Média geral de Matemática dos alunos de Escola Pública 
print('Média Geral de Matemática (Pública): ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_ESCOLA == 2].mean(),2))

Média Geral de Matemática (Pública):  488.57


In [27]:
#Comparação de Médias de Matemática por raça declarada
print('Média Matemática (Não Declarados): ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 0].mean(),2))
print('Média Matemática (Branco)        : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 1].mean(),2))
print('Média Matemática (Preto)         : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 2].mean(),2))
print('Média Matemática (Pardo)         : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 3].mean(),2))
print('Média Matemática (Amarelo)       : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 4].mean(),2))
print('Média Matemática (Indígena)      : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.TP_COR_RACA == 5].mean(),2))

Média Matemática (Não Declarados):  506.64
Média Matemática (Barnco)        :  552.04
Média Matemática (Preto)         :  504.8
Média Matemática (Pardo)         :  508.38
Média Matemática (Amarelo)       :  519.46
Média Matemática (Indígena)      :  486.13


In [36]:
#Comparação de Médias de Matemática por Renda declarada
print('Média Matemática (Sem Renda)                      : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'A'].mean(),2))
print('Média Matemática (Até R$ 998,00)                  : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'B'].mean(),2))
print('Média Matemática (De R$ 2.495,01 até R$ 2.994,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'F'].mean(),2))
print('Média Matemática (De R$ 2.994,01 até R$ 3.992,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'G'].mean(),2))  
print('Média Matemática (De R$ 5.988,01 até R$ 6.986,00) : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'J'].mean(),2))
print('Média Matemática (Mais de R$ 19.960,00)           : ', round(enem_ce['NU_NOTA_MT'].loc[enem_ce.Q006 == 'Q'].mean(),2))

Média Matemática (Sem Renda)                      :  474.23
Média Matemática (Até R$ 998,00)                  :  487.05
Média Matemática (De R$ 2.495,01 até R$ 2.994,00) :  587.22
Média Matemática (De R$ 2.994,01 até R$ 3.992,00) :  589.39
Média Matemática (De R$ 5.988,01 até R$ 6.986,00) :  636.48
Média Matemática (Mais de R$ 19.960,00)           :  691.62


## PARTE III - Carga dos dados
### Realizando Carga de dados no SQlite - Banco de dados Portátil

In [37]:
#Importando biblioteca SQLite
import sqlite3 as lite

In [56]:
#Cria/Conceta Banco de Dados enemce_2019.db e o cursor de navegação interna
con = lite.connect("enemce_2019.db")
cur = con.cursor()

In [57]:
# Deletando Tabela com mesmo nome caso eista
cur.execute("DROP TABLE IF EXISTS enem19")
con.commit()

In [58]:
# criando a tabela (enem19)
cur.execute("""
CREATE TABLE IF NOT EXISTS enem19 (
        NU_INSCRICAO INT,
        SG_UF_RESIDENCIA VARCHAR(2),
        TP_SEXO VARCHAR(1),
        TP_COR_RACA INT,
        TP_ESCOLA INT,
        TP_ENSINO INT,
        NU_NOTA_CN FLOAT,
        NU_NOTA_MT FLOAT,
        Q001 VARCHAR(1),
        Q002 VARCHAR(1),
        Q004 VARCHAR(1),
        Q005 INT,
        Q006 VARCHAR(1) );
""")
con.commit()

In [61]:
#Carregando Dataframe Pandas na Tabela SQLite
cur.executemany("INSERT INTO enem19 VALUES (?,?,?,?,?,?,?,?,?,?,?,?,?);", enem_ce.values.tolist())
con.commit()

In [68]:
#Lendo os Dados do Banco de Dados
cur.execute("SELECT count(*)  FROM enem19;")
for linha in cur.fetchall():
    print(f'Número de Registros: {linha[0]}')

Número de Registros: (222780,)


In [55]:
#Fecha a Conexão com o Banco de Dados
con.close()